In [1]:
pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 66.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import rasterio
from tqdm import tqdm

# Paths
input_root = '/kaggle/input/eurosat-dataset/EuroSATallBands'
output_root = '/kaggle/working/EuroSATallBands_16x16'

orig_size = 64
clip_size = 16
start = (orig_size - clip_size) // 2
end = start + clip_size

os.makedirs(output_root, exist_ok=True)

# First, count total files
all_files = []
for class_folder in os.listdir(input_root):
    class_input_path = os.path.join(input_root, class_folder)
    if not os.path.isdir(class_input_path):
        continue
    for img_name in os.listdir(class_input_path):
        if img_name.lower().endswith('.tif'):
            all_files.append((class_folder, img_name))
total_files = len(all_files)

# Single progress bar for all files
for class_folder, img_name in tqdm(all_files, total=total_files, desc="Processing Images"):
    class_input_path = os.path.join(input_root, class_folder)
    class_output_path = os.path.join(output_root, class_folder)
    os.makedirs(class_output_path, exist_ok=True)
    img_path = os.path.join(class_input_path, img_name)
    with rasterio.open(img_path) as src:
        img = src.read()  # Shape: (bands, 64, 64)
        clipped_img = img[:, start:end, start:end]
        meta = src.meta.copy()
        meta.update({'height': clip_size, 'width': clip_size, 'transform': rasterio.windows.transform(rasterio.windows.Window(start, start, clip_size, clip_size), src.transform)})
        save_path = os.path.join(class_output_path, img_name)
        with rasterio.open(save_path, 'w', **meta) as dst:
            dst.write(clipped_img)

print("Done! All images have been center-cropped to 16x16 and saved to", output_root)

Processing Images: 100%|██████████| 27597/27597 [11:54<00:00, 38.65it/s]

Done! All images have been center-cropped to 16x16 and saved to /kaggle/working/EuroSATallBands_16x16


In [3]:
import shutil

# Define folder to zip and zip file name
folder_to_zip = '/kaggle/working/EuroSATallBands_16x16'
shutil.make_archive(folder_to_zip, 'zip', folder_to_zip)
print('Folder zipped successfully!')

Folder zipped successfully!


In [4]:
from IPython.display import FileLink
FileLink('/kaggle/working/EuroSATallBands_16x16.zip')

/kaggle/working/EuroSATallBands_16x16.zip